## Evaluation Notebook

In [6]:
!pip install xlrd


     |████████████████████████████████| 96 kB 1.9 MB/s eta 0:00:011


In [63]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import re
import os
from spacy import displacy
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

FOODKEEPER_PATH = "data/FoodKeeper-Data.xls"
TRAINING_DATA_PATH = "data/data.csv"
MODEL_PATH = "output/model-last"
TEST_DATA_PATH = "test/test_data.csv"

pd.options.mode.chained_assignment = None

nlp = spacy.load(MODEL_PATH)
food_data = pd.read_excel(FOODKEEPER_PATH, sheet_name = "Product")
training_data = pd.read_csv(TRAINING_DATA_PATH,index_col = False, header = None)
test_data = pd.read_csv(TEST_DATA_PATH)

#write a method to find the top x keywords in the dataset
#loop through and count the specific entities
keywords = ['chicken', 'milk', 'butter', 'cheese']
sampleData = []


    
#update rank tweet to take the counter as a parameter and condense both rankings
def rankTweet(tweet):
    tweetKeywords = []
    doc = nlp(tweet)
    return len(doc.ents)
       
        
    print(tweetKeywords)
    
def findNewKeywords(tweet, keywords):
    foodkeeperKeys = foodKeeperInfo()
    x = tweet.split()
    word = ""
    i = 0
    while i < len(x):
    #for i in range(len(x)):
        z = 1
        if x[i] in foodkeeperKeys:
            word = x[i]
        try:
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in keywords:
                word = foundBiWord
                z = 2
        except:
            pass
        
        try:
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in keywords:
                word = foundTriWord
                z = 3
        except:
            pass
        i += z
        
        if word not in keywords and word != "":
            keywords.append(word)
    #print(keywords)
    
    
def convertToTrainingFormat(tweet, keywords):
    x = tweet.split()
    myEnts = {'entities':[]}
    found = False
    i = 0
    while i < len(x):
        z = 1
        if x[i] in keywords:
            pos = tweet.find(x[i])
            y = (pos, pos + len(x[i]), 'FOOD')
            found = True
            
        try:
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1]) + 1, 'FOOD')
                found = True
                z = 2
        except:
            pass
        try:
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1])+len(x[i+2]) + 2, 'FOOD')
                found = True
                z = 3
        except:
            pass
        
        try:
            if y not in myEnts['entities']:
                myEnts['entities'].append(y)  
        except:
            pass
        #print(z)
        i += z
        #print(i)
        
        
    formatted = (tweet, myEnts)
    #print(formatted)
    if found:
        return formatted
    else: return ()
   
    

print(convertToTrainingFormat('I like chicken and salsa', keywords))

#print(food_data['Keywords'])

def foodKeeperInfo():              
    keywords = []
    for word in food_data['Name']:
        word = word.lstrip()
        word = word.rstrip()

        if word.lower() not in keywords: 
            keywords.append(word.lower())

    #print("Total foodkeeper food names: " + str(len(keywords)))        
    #for element in sorted(keywords):
        #print(element)
        
    return keywords



def preProcess(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    #fix punctuation
    tweet = re.sub('@[a-zA-z0-9]*', '<USERNAME>', tweet)
    tweet = re.sub('https[a-zA-z0-9./:]*', '<URL>', tweet)
    
    return tweet


noEntity= []
               
           
def trainModel(data):
    counter = 0
    while counter < 4:
        print("~~~~~~~~~~~~~~~~~"+str(counter)+"~~~~~~~~~~~~~~~~~")
        nlp = spacy.blank("en") # load a new spacy model
        db = DocBin() # create a DocBin object

        myTweets = []
        #Loop through all the tweets
        for i in range(len(data[0])):
            
            if counter == 0:
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                if x!= ():
                    if len(x[1]['entities']) > 1:
                        #print("Found tweet", x[0])
                        myTweets.append(x)  
        
            else:
                #convert each tweet into spacy training format
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                if x != ():
                    #print(len(x[1]['entities'])
                    #check the ranking of the tweet
                    if rankTweet(x[0]) > 2:
                        #print("Checking rank...")
                        myTweets.append(x)




        for text, annot in tqdm(myTweets): # data in previous format
            doc = nlp.make_doc(text) # create doc object from text
            ents = []
            for start, end, label in annot["entities"]: # add character indexes
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping entity")
                else:
                    ents.append(span)
            doc.ents = ents # label the text with the ents
            db.add(doc)

            db.to_disk("./train.spacy") # save the docbin object

        stream = os.popen('python3 -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy')
        print(stream.read())
        #print("Training model now...")
        eval_model()
        print(keywords)

        for element in myTweets:
            findNewKeywords(element[0], keywords)

        print(keywords)
        counter += 1
        
        
        

def information(data):
    myData = {}
    totalEnt=0
    
    for i in range(len(data[0])):
        doc = nlp(preProcess(data[0][i]))
        if len(doc.ents) == 0:
            noEntity.append(preProcess(data[0][i]))
            
        #print(len(doc.ents))
        if(len(doc.ents) == 4):
            print(doc)
            
        for entity in doc.ents: 
        #print(entity.label_)
            totalEnt+=1
            if(entity.label_ == 'FOOD'):
                if entity.text in myData:
                    myData[entity.text] += 1
                else:
                    myData[entity.text] = 1
                    
    print("Number of entities found: " + str(len(myData)))
    print(totalEnt)
    for i in sorted(myData, key = myData.get, reverse=True):
        print("Entity: " + i, "Count: " + str(myData[i]), "Density: " + str(format(myData[i]/totalEnt, '.2f')), end = "\n")
    
    
    return myData




for i in range(len(test_data['tweet'])):
    test_data['tweet'][i] = preProcess(test_data['tweet'][i])



y = test_data['food'].tolist()
print(nlp.pipe_names)
    
def ent_recognize(text):
    doc = nlp(text)
    displacy.render(doc,style = "ent")
    
def predict(tweet):
    doc = nlp(str(tweet))
    if doc.ents:
        displacy.render(doc,style = "ent")

def returnPrediction(tweet):
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(str(tweet))
    if doc.ents:
        return 1
    else:
        return 0
    
def get_predictions():
    predictions = []
    for tweet in test_data['tweet'].tolist():
        predictions.append(returnPrediction(tweet))
    return predictions
    
def eval_model():
    #nlp = spacy.load(MODEL_PATH)
    predictions = get_predictions()
    print(metrics.confusion_matrix(y,predictions, labels = [1,0]))
    print(metrics.classification_report(y,predictions, labels = [1,0]))
    
def show_tp():
    counter = 0
    tweets = test_data['tweet'].tolist()
    predictions = get_predictions()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 1:
            print("True positives:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_tn():
    counter = 0
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 0:
            print("True Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fn():
    predictions = get_predictions()
    tweets = test_data['tweet']
    counter = 0
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 1:
            print("False Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fp():
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 0:
            print("False Positive:")
            doc = nlp(str(tweets[i]))
            if doc.ents:
                displacy.render(doc,style = "ent")

('I like chicken and salsa', {'entities': [(7, 14, 'FOOD')]})
['tok2vec', 'ner']


## Use the function below to check individual sentences

In [34]:
ent_recognize("My chicken is tasty")

/home/brandon/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


## Use the function below to check model performance on the entire test set

In [25]:
eval_model()

[[ 0 44]
 [ 0 39]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        44
           0       0.47      1.00      0.64        39

    accuracy                           0.47        83
   macro avg       0.23      0.50      0.32        83
weighted avg       0.22      0.47      0.30        83



/home/brandon/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brandon/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brandon/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Use the functions below to see TP, TN, FP, FN respectively

In [2]:
# show_tp()
# show_tn()
# show_fp()
#show_fn()

In [3]:
#training_data.info()

In [4]:
myTrainingdata = information(training_data)

KeyboardInterrupt: 

In [246]:
#Tweets where no entities were found
print("Number of Tweets where no entities were found: ", len(noEntity), '\n')

for i in range(10):
    print(noEntity[i])
    

Number of Tweets where no entities were found:  12716 

<USERNAME> billie is butter?
"the anasa life coco smooth body butter is soooo amazing. i got the yuce scent, which is light and citrusy. the texture is so soft, and not too heavy or oily. it's absolutely perfect. if you love natural, quality body products, get this..." - arielle b.
dairy. all dairy products, like milk, butter, yogurt, and cheese, must come from a kosher animal.
<USERNAME> bread , butter and sugar !!!
i use body butter. but, yes after every bath/shower! it’s so important to moisturize! <URL>
<USERNAME> billie is butter?
as the brand &amp; content lead of <USERNAME>, buzzfeed's black culture vertical, <USERNAME> aims to "push the culture forward."  "life is so short, so while i'm here, i'm going to disrupt some things, shake the table, and make my presence known." <URL>
<USERNAME> like. butter bread
<USERNAME> <USERNAME> <USERNAME> love that video!!!!! so does peanut butter!!!!!!
<USERNAME> <USERNAME> <USERNAME> a b

In [250]:
foodkeeper = foodKeeperInfo()

Total foodkeeper food names: 466
"genuine" maple syrup
aioli
almond butter
almond extract
almond milk
almond oil
almonds
amaranth
anchovies
apple cider
apple cider vinegar
apple juice
apples
applesauce
apricots
artichokes, whole
arugula
asparagus
avocado oil
avocados
baby carrots
bacon
bacon bits
bacon grease
bagel
bagged greens
baking powder
baking soda
balsamic vinegar
bamboo shoots
bananas
barbecue sauce
barley
base
basil
bean sprouts
beans
beans and peas
beef
beef broth/stock/consommé
beets
berries
biscuit or pancake mix
biscuits
bison
black bean sauce
black pepper
blueberries
bok choy
bratwurst
bread
breadcrumbs
broccoli and broccoli raab (rapini)
broth
brussels sprouts
buckwheat
bulgur
butter
butter flavor
buttermilk
cabbage
cajun seasoning blend
cake, brownie, bread mixes
canadian bacon
canned chicken
canned goods
canola oil
cantaloupe
capers
capon
carrot juice
carrots, parsnips
cashew butter
cashews
casseroles
cauliflower
caviar
celery
celery root
cereal
cereal or granola bars


In [254]:
##     Information about the different datasets
x = list(myTrainingdata)

print("Training data entities: ",len(x))
print("Foodkeeper entities: ", len(foodkeeper))


diff = list(set(x) & set(foodkeeper))
print("Union between foodkeeper and training data: " + str(len(diff)))
#for elem in sorted(diff):
    #print(elem)

onlyTraining = list(set(x) - set(foodkeeper))
onlyFoodkeeper = list(set(foodkeeper) - set(x))

        
print("Unique training data entities: ", len(onlyTraining))
print("Unique foodkeeper data entities: ",len(onlyFoodkeeper), '\n')

print("~~~~~~~~~~~~Only Training~~~~~~~~~~~~")
for element in sorted(onlyTraining):
    print(element)

print("~~~~~~~~~~~~Only Foodkeeper~~~~~~~~~~~~")
for element in sorted(onlyFoodkeeper):
    print(element)

Training data entities:  409
Foodkeeper entities:  466
Union between foodkeeper and training data: 223
Unique training data entities:  186
Unique foodkeeper data entities:  243 

~~~~~~~~~~~~Only Training~~~~~~~~~~~~
0.15
2.they
4.99
699,615,065,551
abstract
acai
acerola
activation
adored
agitating
ahora
alejandro
algae
alvarrez
approached
aslan
athletes
attracted
azul
bffs
blues
bootcamps
braves
brazils
bulborb
cafe
calamagrostis
cameron
capricorn
capsaicin
carafe
carbseedling
cested
chickenbreast
chunibyo
citeh
clashes
coconutsㅋㅋㅋ
collapse
commodified
corn‼️
coupled
creamery
cuddling
curbside
d175
dalmunach
dani
decadent!⁣
delusional
derwin
dinnerly
doggos
drains
dram
dranks
driveable
enrolled
erika
european
exporters
extent
firewild
frickin
fuckn
gals
gasc
glenbrook
goddddddd
goldening
grained
grann
griswald
gulf
hachinohe
hamin
hank
hauck
helga
hibby
hocking
howd
iffco
indians
insects
karedok
kiddush
kiita
kirkby
knabe
krogan
latino
libya
lindsay
lure
mach
malnourished
marathi
mato

In [64]:
trainModel(training_data)
    

~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~


 35%|███▍      | 80/231 [00:00<00:00, 239.92it/s]

Skipping entity
Skipping entity
Skipping entity


 77%|███████▋  | 178/231 [00:01<00:00, 136.46it/s]

Skipping entity


...

Skipping entity


100%|█████████▉| 2781/2787 [03:35<00:01,  5.20it/s]

Skipping entity


100%|██████████| 2787/2787 [03:36<00:00, 12.87it/s]


ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     20.67    0.08    0.07    0.09    0.00
  0     200         59.29   1709.11   83.40   77.75   89.92    0.83
  0     400         41.47    945.60   79.36   85.09   74.35    0.79
  0     600         28.63    985.01   88.09   82.96   93.90    0.88
  0     800         27.02   1109.96   89.33   84.97   94.17    0.89
  1    1000         36.54   1259.77   90.80   86.95   95.01    0.91
  1    1200         45.23   1342.01   91.59   86.79   96.96    0.92
  2    1400         67.94   1524.37   93.34   90.71   96.13    0.93
  2    1600         80.05   1529.95   93.98   91.98   96.07    0.94


In [67]:
nlp = spacy.load(MODEL_PATH)
ent_recognize("My friend likes to drink milk and eat fish and chicken")
ent_recognize("My friend is a chicken")
ent_recognize("A chicken is a wild animal.")
ent_recognize("The chicken is a wild animal.")

#rankTweet("sugar")
#eval_model()


In [7]:
#eval_model()
# show_tp()
# show_tn()
# show_fp()
#show_fn()


In [25]:
rankTweet("Hello chicken")

1

In [31]:
print(keywords)

['chicken', 'milk']
